In [156]:
import numpy as np
import cvxopt
import struct
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

### Control_horizon=1, planning_horizon=1

In [293]:
del_t = 0.1
v_init = 10
w_init = 0.1
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            

    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        while (np.linalg.norm(self.pos-self.goal)>0.5):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            diff_v = 999
            diff_w = 999
#             xgi = self.pos[0]
#             ygi = self.pos[1]
            tol = 1e-4
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                # vi = self.v, wi = self.w
#                 theta_new += self.w*del_t
#                 xgi += (self.v*del_t*np.cos(theta_new) - self.goal[0])
#                 ygi += (self.v*del_t*np.sin(theta_new) - self.goal[1])
                w_guess = 0.1
                v_guess = 10
                x = del_t*np.cos(self.theta+w_guess*del_t)
                a = -v_guess*del_t**2*np.sin(self.theta+w_guess*del_t)
                y = del_t*np.sin(self.theta+w_guess*del_t)
                b = v_guess*del_t**2*np.cos(self.theta+w_guess*del_t)
                kx = self.pos[0]+v_guess*del_t**2*w_guess*np.sin(self.theta+w_guess*del_t) - self.goal[0]
                ky = self.pos[1]-v_guess*del_t**2*w_guess*np.cos(self.theta+w_guess*del_t) - self.goal[1]
                arrx = np.array([[x],[a]])
                arry = np.array([[y],[b]])
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[0,-1],[-1,0],[1,0],[0,1]])
                h = cvxopt.matrix(np.array([self.wmax,0,self.vmax,self.wmax]),tc='d')
                print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0] #vd
                w_new = sol['x'][1] #wd
                print ("Optimizer op:",v_new,w_new)
#                 print (v_new,w_new)
                diff_v = v_new - self.v
                diff_w = w_new - self.w
                print (diff_v,diff_w)
                self.v = v_new #vd+vi
                self.w = w_new #wd+wi
            print ("##########End of Optimization#############")
            print ("new v: ",self.v,"new w: ",self.w)
            print ("Opti complete")
#             self.vplot.append(self.v)
#             self.wplot.append(self.w)
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [294]:
bot = agent(2,np.array([0,0]),np.array([50,50]),np.pi/4,20,0.5)
bot.opt_traj()

Theta:  0.7853981633974483
start v:  10 start w:  0.1
[[ 0.06928242]
 [-0.07211066]] [[0.07211066]
 [0.06928242]] -49.29278845824543 -49.292785865613666
Optimizer op: 19.999999627396868 -0.4999857434441809
9.999999627396868 -0.5999857434441809
[[ 0.06928242]
 [-0.07211066]] [[0.07211066]
 [0.06928242]] -49.29278845824543 -49.292785865613666
Optimizer op: 19.999999627396868 -0.4999857434441809
0.0 0.0
##########End of Optimization#############
new v:  19.999999627396868 new w:  -0.4999857434441809
Opti complete
0 0.7953981633974483 19.999999627396868 -0.4999857434441809 [0.7000004761807905, 0.7141423761034396]
Theta:  0.7953981633974483
[[ 0.07279977]
 [-0.06855796]] [[0.06855796]
 [0.07279977]] -47.82350988778882 -47.93660664688023
Optimizer op: 19.999999728670616 0.4999925394868552
1.0127374849844273e-07 0.9999782829310361
[[ 0.07279977]
 [-0.06855796]] [[0.06855796]
 [0.07279977]] -47.82350988778882 -47.93660664688023
Optimizer op: 19.999999728670616 0.4999925394868552
0.0 0.0
######

[[ 0.07279853]
 [-0.06855927]] [[0.06855927]
 [0.07279853]] -27.736183607126335 -28.442776161689313
Optimizer op: 19.999998511310935 0.4999770283392202
7.909686168261487e-07 0.999887371427062
[[ 0.07279853]
 [-0.06855927]] [[0.06855927]
 [0.07279853]] -27.736183607126335 -28.442776161689313
Optimizer op: 19.999998511310935 0.4999770283392202
0.0 0.0
##########End of Optimization#############
new v:  19.999998511310935 new w:  0.4999770283392202
Opti complete
15 0.7454175388279761 19.999998511310935 0.4999770283392202 [22.256960465968074, 21.56450369176149]
Theta:  0.7454175388279761
[[ 0.06928119]
 [-0.07211184]] [[0.07211184]
 [0.06928119]] -26.335851894810638 -27.014115871532415
Optimizer op: 19.999998009117377 -0.4994789795071814
-5.021935578497505e-07 -0.9994560078464017
[[ 0.06928119]
 [-0.07211184]] [[0.07211184]
 [0.06928119]] -26.335851894810638 -27.014115871532415
Optimizer op: 19.999998009117377 -0.4994789795071814
0.0 0.0
##########End of Optimization#############
new v:  19

[[ 0.07018997]
 [-0.07122758]] [[0.07122758]
 [0.07018997]] -8.126048740355607 -8.472040713448457
Optimizer op: 19.999999957024322 0.4999731182512126
9.317242444240037e-08 0.9999726006265695
[[ 0.07018997]
 [-0.07122758]] [[0.07122758]
 [0.07018997]] -8.126048740355607 -8.472040713448457
Optimizer op: 19.999999957024322 0.4999731182512126
0.0 0.0
##########End of Optimization#############
new v:  19.999999957024322 new w:  0.4999731182512126
Opti complete
29 0.782735261720251 19.999999957024322 0.4999731182512126 [41.866828501505815, 41.53497828338153]
Theta:  0.782735261720251
[[ 0.06654255]
 [-0.07464642]] [[0.07464642]
 [0.06654255]] -6.779993276687776 -6.992130452413583
Optimizer op: 19.999999864861966 -0.4999978374673318
-9.216235596909428e-08 -0.9999709557185443
[[ 0.06654255]
 [-0.07464642]] [[0.07464642]
 [0.06654255]] -6.779993276687776 -6.992130452413583
Optimizer op: 19.999999864861966 -0.4999978374673318
0.0 0.0
##########End of Optimization#############
new v:  19.99999986

### Control_horizon = 1, planning_horizon = p

In [306]:
del_t = 0.1
planning_horizon = 5
v_guess = 10*np.ones((planning_horizon))
w_guess = 0.1*np.ones((planning_horizon))
v_init = v_guess[0]
w_init = w_guess[0]
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class mpc_agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.v_guess = v_guess
        self.w_guess = w_guess
        self.prev_v = v_guess
        self.prev_w = w_guess
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            
    def p_constructor(self):
#         print (self.w_guess)
        wg = np.cumsum(self.w_guess)
        p_theta = self.theta+wg*del_t
        
        xs = del_t*np.cos(p_theta)
        qx = np.sin(p_theta)
        bs = -self.v_guess*del_t**2*qx
        brev = bs[::-1]
        dterms = np.cumsum(brev)
        dterms = dterms[::-1]
        arrx = np.concatenate((xs, dterms), axis=None)
        ksx = bs*self.w_guess
        kx = np.sum(ksx) + self.pos[0] - self.goal[0]
        
        ys = del_t*np.sin(p_theta)
        qy = np.cos(p_theta)
        by = self.v_guess*del_t**2*qy
        byrev = by[::-1]
        dyterms = np.cumsum(byrev)
        dyterms = dyterms[::-1]
        arry = np.concatenate((ys, dyterms), axis=None)
        ksy = by*self.w_guess
        ky = np.sum(ksy) + self.pos[1] - self.goal[1]
        
        
        return arrx,arry,kx,ky
        
    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        
        while (np.linalg.norm(self.pos-self.goal)>0.5):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            diff_v = 999
            diff_w = 999
            tol = 1e-4
            
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                arrx,arry,kx,ky = self.p_constructor()
                arrx = arrx.reshape((-1,1))
                arry = arry.reshape((-1,1))
#                 print (arrx.shape,arry.shape,kx.shape,ky.shape)
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.eye(2*planning_horizon)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
#                 g_mat[:planning_horizon,:planning_horizon] = np.eye(planning_horizon)
#                 print (g_mat)
#                 g_mat = np.array([[0,-1],[-1,0],[1,0],[0,1]])
#                 h_mat = np.concatenate((self.wmax*np.ones(2*planning_horizon),np.zeros(2*planning_horizon),self.vmax*np.ones(2*planning_horizon),self.wmax*np.ones(2*planning_horizon)),axis=None)
#                 h = cvxopt.matrix(h_mat,tc='d')
#                 h_mat = np.array([self.wmax,0,self.vmax,self.wmax])
                h_mat = np.concatenate((self.vmax*np.ones(planning_horizon),self.wmax*np.ones(planning_horizon),np.zeros(planning_horizon),self.wmax*np.ones(planning_horizon)),axis=None)
                h = cvxopt.matrix(h_mat,tc='d')
#                 print ("h_mat:",h_mat)
#                 print ("p_mat:",p_mat)
#                 print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
#                 print (p_mat.shape,q_mat.shape,g_mat.shape,h_mat.shape)
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][:planning_horizon] #vd
                w_new = sol['x'][planning_horizon:] #wd
#                 print ("Optimizer op:",v_new,w_new)
                diff_v = v_new[0] - self.v
                diff_w = w_new[0] - self.w
#                 print (diff_v,diff_w)
                self.v = v_new[0]
                self.w = w_new[0]
                self.prev_v = np.array(v_new)
                self.prev_w = np.array(w_new)
#                 print (self.v,self.w)
#                 self.v_guess = self.prev_v[1:]
#                 self.v_guess = np.concatenate((self.v_guess,np.array([[self.v]])),axis=0)
#                 self.v_guess = self.v_guess.reshape((-1,))
#                 self.w_guess = self.prev_w[1:]
#                 self.w_guess = np.concatenate((self.w_guess,np.array([[self.w]])),axis=0)
#                 self.w_guess = self.w_guess.reshape((-1,))
                self.v_guess = 10*np.ones((planning_horizon))
                self.w_guess = 0.1*np.ones((planning_horizon))
#                 print (self.v_guess,self.w_guess)
                
            print ("##########End of Optimization#############")
            print ("new v: ",self.v,"new w: ",self.w)
            print ("Opti complete")
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
#         ax.plot(self.prev_v,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [307]:
bot1 = mpc_agent(2,np.array([0,0]),np.array([80,50]),0,20,0.5)
bot1.opt_traj()

Theta:  0
start v:  10.0 start w:  0.1
##########End of Optimization#############
new v:  19.9999948028806 new w:  0.4999986074936816
Opti complete
0 0.010000000000000002 19.9999948028806 0.4999986074936816 [0.9999500004166653, 0.009999833334166666]
Theta:  0.010000000000000002
##########End of Optimization#############
new v:  19.999996271557457 new w:  0.4999989503993793
Opti complete
1 0.05999986074936816 19.999996271557457 0.4999989503993793 [2.9963505782103628, 0.12992753712900504]
Theta:  0.05999986074936816
##########End of Optimization#############
new v:  19.999995819877896 new w:  0.4999984893229885
Opti complete
2 0.1099997557893061 19.999995819877896 0.4999984893229885 [4.984262457150937, 0.34948361240381315]
Theta:  0.1099997557893061
##########End of Optimization#############
new v:  19.999995402886178 new w:  0.4999980311005985
Opti complete
3 0.15999960472160496 19.999995402886178 0.4999980311005985 [6.958716737179041, 0.6681191785762554]
Theta:  0.15999960472160496
###

##########End of Optimization#############
new v:  19.99999973612926 new w:  0.499967418745197
Opti complete
36 0.6099894474789853 19.99999973612926 0.499967418745197 [61.96098895431459, 36.24710839422151]
Theta:  0.6099894474789853
##########End of Optimization#############
new v:  19.999999829054815 new w:  -0.49988825187091457
Opti complete
37 0.6599861893535051 19.999999829054815 -0.49988825187091457 [63.54099033139304, 37.47332026126648]
Theta:  0.6599861893535051
##########End of Optimization#############
new v:  19.999999816006277 new w:  0.4999993431396207
Opti complete
38 0.6099973641664136 19.999999816006277 0.4999993431396207 [65.18028937303337, 38.61905085075906]
Theta:  0.6099973641664136
##########End of Optimization#############
new v:  19.999999531482008 new w:  -0.29161360203242687
Opti complete
39 0.6599972984803757 19.999999531482008 -0.29161360203242687 [66.76027713418136, 39.84528027506149]
Theta:  0.6599972984803757
##########End of Optimization#############
new v

In [74]:
wg = np.array([[5,6,7]])
print (wg)
c = np.cumsum(wg)
c[::-1]

[[5 6 7]]


array([18, 11,  5])

In [56]:
np.ones((2,1))

array([[1.],
       [1.]])

In [129]:
a = np.array([1,0,3])
b = np.array([[4,5,6,1]])
# np.concatenate((a, b), axis=None)

In [62]:
b[::-1]

array([6, 5, 4])

In [66]:
np.sum(a)

4

In [70]:
np.concatenate((np.zeros(2),np.ones(2)),axis=None)

array([0., 0., 1., 1.])

In [71]:
np.array([a,b])

array([[1, 0, 3],
       [4, 5, 6]])

In [87]:
a.reshape((-1,1))

array([[1],
       [0],
       [3]])

In [113]:
c = np.zeros((4,4))

In [114]:
c[:2,:2] = np.eye(2)
c

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [130]:
np.concatenate((c,b),axis = 0)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [4., 5., 6., 1.]])

In [131]:
np.concatenate((c,-np.eye(4)),axis=0)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.]])

In [153]:
print (a)
np.insert(a,2,4)

[1 0 3]


array([1, 0, 4, 3])

In [198]:
np.cumsum(w_guess)

array([0.1, 0.2, 0.3, 0.4, 0.5])

In [278]:
v_guess.shape

(5,)